In [ ]:
!pip install langchain

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
!setsid nohup ollama serve > run_serve.txt 2>&1 & 

In [ ]:
!ollama list

In [11]:
!setsid ollama run gemma:2b > run_gemma_2b.txt 2>&1 &

## LLM + Prompt

In [12]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma:2b")

llm.invoke("how can langsmith help with testing?")

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | llm

chain.invoke({"input": "how can langsmith help with testing?"})

In [17]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

## RAG Retrieval Chain

In [ ]:
!pip install beautifulsoup4
!pip install faiss-cpu

In [21]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
embeddings = OllamaEmbeddings(model="gemma:2b")
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}
    """)

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke(
    {"input": "how can langsmith help with testing?"})

print(response["answer"])

In [29]:
# small document for comparison
# llm could not answer the question based on the small document

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain_core.documents import Document
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

## Conversation Retrieval Chain

In [33]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

from langchain.chains import create_history_aware_retriever

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"), 
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [35]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"), 
    AIMessage(content="Yes!")
]

retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})